In [209]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hexstorypoints/Hexaview1.csv
/kaggle/input/hexstorypoints/RapidX.csv
/kaggle/input/hexstorypoints/Augmented_Storypointsdata (1).csv
/kaggle/input/hexstorypoints/Hexaview2.csv
/kaggle/input/hexstorypoints/ATOP.csv
/kaggle/input/hexstorypoints/ATOP - JUMBO.csv
/kaggle/input/hexstorypoints/Test-SP (1).csv
/kaggle/input/hexstorypoints/Coco.csv
/kaggle/input/hexstorypoints/Tensai.csv


In [210]:
merged_aug = pd.read_csv('../input/hexstorypoints/Augmented_Storypointsdata (1).csv', index_col=0)
stp_test = pd.read_csv('../input/hexstorypoints/Test-SP (1).csv', index_col=0)

In [211]:
df3=pd.DataFrame()
df3['Y']= merged_aug['Story Points']
df3['X']=merged_aug['Cleaned_desc']

In [212]:
merged_data = df3.loc[df3['Y'].isin([2,3,5,1,8])]
stp_test = stp_test.loc[stp_test['Story Points'].isin([2,3,5,1,8])]

In [213]:
merged_data['Y'].value_counts()

2    456
3    452
5    344
1    176
8     88
Name: Y, dtype: int64

In [214]:
df3['word_count'] = df3['X'].apply(lambda text : len(text.split(" ")))

In [215]:
df3['word_count'].mean()

48.236434108527135

In [216]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

merged_data['encoded_labels'] = le.fit_transform(merged_data['Y'])
stp_test['encoded_labels'] = le.transform(stp_test['Story Points'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [217]:
merged_data.shape
merged_data.head()

,Y,X,encoded_labels
0,2,u vod changes for drop downs core dev u vod ch...,1
1,2,u vod changes for drop downs core dev u vod ch...,1
2,2,u vod changes for drop downs core dev u vod ch...,1
3,5,drt drt ui new angular application setup as pe...,3
4,5,drt drt ui new angular application setup as pe...,3


In [218]:
stp_test.head()

,Cleaned_desc,Story Points,encoded_labels
279,rest api - test user stories for get api with...,1,0
364,latest code deployment to aws url latest code...,8,4
85,external tool integration - update connection...,2,1
386,jenkins new metrics - deployment time by comp...,5,3
433,define the purpose of the mvp what are our go...,1,0


In [219]:
X_train = merged_data.X
X_test = stp_test.Cleaned_desc
y_train = merged_data.encoded_labels
y_test = stp_test.encoded_labels

In [220]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1516,)
(1516,)
(95,)
(95,)


In [221]:
#!pip install transformers
!pip install pytorch-transformers
#!pip install transformers==3

In [222]:
import transformers

class config:
    DEVICE = "cuda"
    MAX_LEN = 64
    TRAIN_BATCH_SIZE = 16
    VALID_BATCH_SIZE = 4
    EPOCHS = 20
    #BERT_PATH = "bert-large-uncased"
    BERT_PATH = "bert-base-uncased"
    MODEL_PATH = "/root/docker_data/model.bin"
    MODEL_OUTPUT_PATH = "./spp_model.bin"
    TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)
    #TOKENIZER = transformers.BertTokenizer.from_pretrained('bert-large-uncased')

In [223]:
import torch


class BERTDataset:
    def __init__(self, review, target):
        self.review = review
        self.target = target
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN

    def __len__(self):
        return len(self.review)

    def __getitem__(self, item):
        review = str(self.review[item])
        review = " ".join(review.split())

        inputs = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(self.target[item], dtype=torch.long),
        }

In [224]:
import torch.nn as nn

class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH, return_dict=False)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 5)
        #self.out = nn.Linear(1024, 5)

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        bo = self.bert_drop(o2)
        output = self.out(bo)
        return output

In [225]:
import torch
import torch.nn as nn
from tqdm import tqdm


def loss_fn(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets)


def train_fn(data_loader, model, optimizer, device):
    model.train()

    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]

        ids = ids.to(device)
        token_type_ids = token_type_ids.to(device)
        mask = mask.to(device)
        targets = targets.to(device)

        
        #outputs = model(ids, attention_mask=mask, token_type_ids=token_type_ids, labels=targets)
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)#, labels=targets)
        #loss = loss_fn(outputs.logits, targets)
        print(type(outputs))
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        



def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device)
            token_type_ids = token_type_ids.to(device)
            mask = mask.to(device,)
            targets = targets.to(device)

            #outputs = model(input_ids=ids, attention_mask=mask, token_type_ids=token_type_ids)
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            sft=torch.nn.Softmax(dim=1)(outputs)#.logits)
            prediction=torch.argmax(sft, dim=1)
            fin_outputs.extend(prediction.cpu().detach().numpy().tolist())
            #fin_outputs.extend(torch.softmax(outputs).cpu().detach().numpy().tolist())
            
    return fin_outputs, fin_targets

In [226]:
device = torch.device(config.DEVICE)
#from transformers import BertForSequenceClassification

#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 5)
model = BERTBaseUncased()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [227]:
train_dataset = BERTDataset(
    review=X_train.values, target=y_train.values
)

valid_dataset = BERTDataset(
    review=X_test.values, target=y_test.values
)

In [228]:
print(len(train_dataset))
print(len(valid_dataset))

1516
95


In [229]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)

train_data_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=config.TRAIN_BATCH_SIZE,
    worker_init_fn=seed_worker,
    generator=g
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=config.VALID_BATCH_SIZE,
    worker_init_fn=seed_worker,
    generator=g
)

In [230]:
print(len(train_data_loader))
print(len(valid_data_loader))

95
24


In [231]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [232]:
valid_size = len(valid_dataset)
valid_size

95

In [233]:
from sklearn.metrics import accuracy_score
best_accuracy = 0
best_epoch = 0

for epoch in range(config.EPOCHS):
    train_fn(train_data_loader, model, optimizer, device)
    outputs, targets = eval_fn(valid_data_loader, model, device)
    print("Epoch#:", epoch)
    print("Accuracy:", np.sum(np.array(outputs)==np.array(targets))/valid_size*100)
    #outputs = np.array(outputs) >= 0.4
    accuracy = sklearn.metrics.accuracy_score(targets, outputs)
    print(f"Accuracy Score = {accuracy}")
    #best_accuracy = accuracy
    print("Previous best accuracy:", best_accuracy)
    #torch.save(model.state_dict(), config.MODEL_OUTPUT_PATH)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_epoch = epoch
        print("Current best accuracy:", best_accuracy)
        print("Best Epoch#:", best_epoch)
        torch.save(model.state_dict(), config.MODEL_OUTPUT_PATH)
        print("Model exported from Epoch:", epoch)
    print("Final Best accuracy:", best_accuracy)
    print("Final Best epoch:", best_epoch)
    

  0%|          | 0/95 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<class 'torch.Tensor'>


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
  2%|▏         | 2/95 [00:00<00:17,  5.42it/s]

<class 'torch.Tensor'>
<class 'torch.Tensor'>


  4%|▍         | 4/95 [00:00<00:15,  6.07it/s]

...

<class 'torch.Tensor'>
<class 'torch.Tensor'>


100%|██████████| 95/95 [00:14<00:00,  6.73it/s]


<class 'torch.Tensor'>


100%|██████████| 24/24 [00:00<00:00, 46.38it/s]

Epoch#: 19
Accuracy: 84.21052631578947
Accuracy Score = 0.8421052631578947
Previous best accuracy: 0.8421052631578947
Final Best accuracy: 0.8421052631578947
Final Best epoch: 8
